In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.rcParams['figure.figsize'] = 9, 6
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import glob

N = 5000
n_min = 100

filenames = glob.glob('./data/survey_lcs/*')
lengths = {f: sum(1 for line in open(f)) for f in filenames}
filenames = [f for f in filenames if lengths[f] >= n_min]
filenames = sorted(filenames, key=lambda f: lengths[f])

X_list = [np.loadtxt(f, delimiter=',') for f in filenames[:N]]

In [13]:
from keras import backend as K
from keras.preprocessing.sequence import pad_sequences

import keras_util as ku
from autoencoder import uneven_gru_autoencoder

model_type = 'gru'
size = 64
num_layers = 2
drop_frac = 0.5
lr = 0.001
nb_epoch = 100
batch_size = 100
loss = 'mse'
sim_type = 'survey_lcs'

model_dict = {'gru': uneven_gru_autoencoder}
X = pad_sequences(X_list, value=-1., dtype='float')
model = model_dict[model_type](input_len=X.shape[-1], aux_input_len=2, n_step=X.shape[1],
                               size=size, num_layers=num_layers, drop_frac=drop_frac)
                                    
run = "{}_{:03d}_x{}_{:1.0e}_drop{}".format(model_type, size, num_layers, lr,
                                            int(100 * drop_frac)).replace('e-', 'm')
if 'conv' in run:
    run += '_f{}'.format(nfilter)

history = ku.train_and_log({'main_input': X, 'aux_input': X[:, [0, 2]]}, X[:, 1:2],
                            run, model, lr=lr, nb_epoch=nb_epoch, batch_size=batch_size,
                           loss=loss, sim_type=sim_type)

[]
/Users/brettnaul/Dropbox/Documents/timeflow/keras_logs/survey_lcs/gru_064_x2_1m03_drop50


Exception: Error when checking model input: expected aux_input to have shape (None, 4108, 2) but got array with shape (5000, 2, 3)